# 59-WebSocket实时通信技术

## 📚 用途说明

**学习目标**：
- 掌握WebSocket协议原理和实现方式
- 学会使用FastAPI构建WebSocket应用
- 理解实时通信的数据处理和状态管理
- 能够实现聊天室、实时通知等应用场景

**前置要求**：
- 已完成FastAPI和RESTful API学习
- 熟练掌握异步编程基础
- 了解HTTP协议和Web基础知识
- 理解前后端交互的基本原理

**与LangChain关联**：
- WebSocket支持LangChain的实时对话流
- 实时通信提升用户交互体验
- 为AI助手提供即时响应能力
- 支持多用户协作和共享会话

---

## 🔢 知识点覆盖

### 7.4 WebSocket实时通信技术 [⭐⭐进阶]
**知识点说明**：WebSocket实时通信技术是构建现代Web应用的核心技能，对于LangChain的实时对话和交互功能具有重要意义。

**学习要求**：
- 掌握WebSocket协议原理和实现方式
- 学会使用FastAPI构建WebSocket应用
- 理解实时通信的数据处理和状态管理
- 能够实现聊天室、实时通知等应用场景

**案例要求**：
- 实现完整的WebSocket通信系统
- 构建实时聊天室应用
- 实现实时通知和状态同步
- 验证点：能独立开发实时通信功能

In [ ]:
print("🌐 WebSocket实时通信技术:")
print("=" * 50)

# 导入必要的库
import asyncio
import json
import time
import datetime
import uuid
import weakref
from typing import List, Dict, Any, Optional, Set
from dataclasses import dataclass, asdict, field
from enum import Enum
from collections import defaultdict, deque

# FastAPI相关
from fastapi import FastAPI, WebSocket, WebSocketDisconnect, HTTPException
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel

print(f"✅ Python版本: {__import__('sys').version}")
print(f"✅ 当前时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. WebSocket基础概念
print(f"📝 1. WebSocket基础概念:")

# 1.1 WebSocket消息类型
print(f"\n   🔍 1.1 WebSocket消息类型:")

class MessageType(Enum):
    """WebSocket消息类型"""
    CONNECT = "connect"      # 连接消息
    DISCONNECT = "disconnect"  # 断开消息
    CHAT = "chat"            # 聊天消息
    NOTIFICATION = "notification"  # 通知消息
    HEARTBEAT = "heartbeat"  # 心跳消息
    STATUS = "status"        # 状态消息

@dataclass
class WebSocketMessage:
    """WebSocket消息结构"""
    type: MessageType
    data: Dict[str, Any]
    timestamp: float = field(default_factory=time.time)
    sender_id: Optional[str] = None
    receiver_id: Optional[str] = None
    room_id: Optional[str] = None

    def to_json(self) -> str:
        """转换为JSON字符串"""
        return json.dumps({
            "type": self.type.value,
            "data": self.data,
            "timestamp": self.timestamp,
            "sender_id": self.sender_id,
            "receiver_id": self.receiver_id,
            "room_id": self.room_id
        })
    
    @classmethod
    def from_json(cls, json_str: str) -> 'WebSocketMessage':
        """从JSON字符串创建消息"""
        data = json.loads(json_str)
        return cls(
            type=MessageType(data["type"]),
            data=data["data"],
            timestamp=data.get("timestamp", time.time()),
            sender_id=data.get("sender_id"),
            receiver_id=data.get("receiver_id"),
            room_id=data.get("room_id")
        )

# 1.2 连接管理器
print(f"\n   🔗 1.2 连接管理器:")

class ConnectionManager:
    """WebSocket连接管理器"""
    
    def __init__(self):
        # 活跃连接 {connection_id: websocket}
        self.active_connections: Dict[str, WebSocket] = {}
        
        # 用户连接映射 {user_id: connection_id}
        self.user_connections: Dict[str, str] = {}
        
        # 房间连接 {room_id: set(connection_id)}
        self.room_connections: Dict[str, Set[str]] = defaultdict(set)
        
        # 连接信息 {connection_id: connection_info}
        self.connection_info: Dict[str, Dict[str, Any]] = {}
        
        # 消息历史
        self.message_history: deque = deque(maxlen=1000)
        
        # 统计信息
        self.stats = {
            "total_connections": 0,
            "active_connections": 0,
            "messages_sent": 0,
            "messages_received": 0
        }
    
    async def connect(self, websocket: WebSocket, user_id: str = None, room_id: str = None) -> str:
        """接受WebSocket连接"""
        await websocket.accept()
        
        connection_id = str(uuid.uuid4())
        
        # 保存连接
        self.active_connections[connection_id] = websocket
        
        # 保存用户映射
        if user_id:
            self.user_connections[user_id] = connection_id
        
        # 加入房间
        if room_id:
            self.room_connections[room_id].add(connection_id)
        
        # 保存连接信息
        self.connection_info[connection_id] = {
            "user_id": user_id,
            "room_id": room_id,
            "connected_at": time.time(),
            "last_heartbeat": time.time()
        }
        
        # 更新统计
        self.stats["total_connections"] += 1
        self.stats["active_connections"] += 1
        
        print(f"      🎉 新连接建立: {connection_id} (用户: {user_id}, 房间: {room_id})")
        
        return connection_id
    
    async def disconnect(self, connection_id: str):
        """断开WebSocket连接"""
        if connection_id not in self.active_connections:
            return
        
        # 获取连接信息
        info = self.connection_info.get(connection_id, {})
        user_id = info.get("user_id")
        room_id = info.get("room_id")
        
        # 从房间移除
        if room_id and connection_id in self.room_connections[room_id]:
            self.room_connections[room_id].remove(connection_id)
            
            # 如果房间为空，删除房间
            if not self.room_connections[room_id]:
                del self.room_connections[room_id]
        
        # 移除用户映射
        if user_id and self.user_connections.get(user_id) == connection_id:
            del self.user_connections[user_id]
        
        # 移除连接
        del self.active_connections[connection_id]
        del self.connection_info[connection_id]
        
        # 更新统计
        self.stats["active_connections"] -= 1
        
        print(f"      👋 连接断开: {connection_id} (用户: {user_id}, 房间: {room_id})")
    
    async def send_personal_message(self, message: str, connection_id: str):
        """发送个人消息"""
        if connection_id in self.active_connections:
            websocket = self.active_connections[connection_id]
            await websocket.send_text(message)
            self.stats["messages_sent"] += 1
            return True
        return False
    
    async def send_user_message(self, message: str, user_id: str):
        """发送用户消息"""
        connection_id = self.user_connections.get(user_id)
        if connection_id:
            return await self.send_personal_message(message, connection_id)
        return False
    
    async def broadcast(self, message: str, room_id: str = None):
        """广播消息"""
        if room_id:
            # 广播到指定房间
            connections = self.room_connections.get(room_id, set())
        else:
            # 广播到所有连接
            connections = set(self.active_connections.keys())
        
        # 并发发送
        tasks = []
        for connection_id in connections:
            if connection_id in self.active_connections:
                task = self.send_personal_message(message, connection_id)
                tasks.append(task)
        
        if tasks:
            await asyncio.gather(*tasks, return_exceptions=True)
    
    async def send_message_object(self, message: WebSocketMessage):
        """发送消息对象"""
        json_str = message.to_json()
        
        if message.receiver_id:
            # 发送给指定用户
            await self.send_user_message(json_str, message.receiver_id)
        elif message.room_id:
            # 广播到房间
            await self.broadcast(json_str, message.room_id)
        else:
            # 广播到所有连接
            await self.broadcast(json_str)
        
        # 保存消息历史
        self.message_history.append(message)
        self.stats["messages_sent"] += 1
    
    def get_connection_list(self) -> List[Dict[str, Any]]:
        """获取连接列表"""
        connections = []
        for connection_id, info in self.connection_info.items():
            connections.append({
                "connection_id": connection_id,
                **info
            })
        return connections
    
    def get_room_list(self) -> List[Dict[str, Any]]:
        """获取房间列表"""
        rooms = []
        for room_id, connections in self.room_connections.items():
            rooms.append({
                "room_id": room_id,
                "connection_count": len(connections),
                "connections": list(connections)
            })
        return rooms
    
    def get_stats(self) -> Dict[str, Any]:
        """获取统计信息"""
        return {
            **self.stats,
            "room_count": len(self.room_connections),
            "user_count": len(self.user_connections),
            "message_history_count": len(self.message_history)
        }

# 1.3 消息处理器
print(f"\n   📨 1.3 消息处理器:")

class MessageHandler:
    """WebSocket消息处理器"""
    
    def __init__(self, connection_manager: ConnectionManager):
        self.connection_manager = connection_manager
        self.handlers = {
            MessageType.CHAT: self._handle_chat,
            MessageType.NOTIFICATION: self._handle_notification,
            MessageType.HEARTBEAT: self._handle_heartbeat,
            MessageType.STATUS: self._handle_status
        }
    
    async def handle_message(self, message: WebSocketMessage, connection_id: str):
        """处理接收到的消息"""
        handler = self.handlers.get(message.type)
        if handler:
            await handler(message, connection_id)
        else:
            print(f"      ⚠️ 未知消息类型: {message.type}")
    
    async def _handle_chat(self, message: WebSocketMessage, connection_id: str):
        """处理聊天消息"""
        # 获取发送者信息
        sender_info = self.connection_manager.connection_info.get(connection_id, {})
        user_id = sender_info.get("user_id", "匿名用户")
        
        # 创建响应消息
        response = WebSocketMessage(
            type=MessageType.CHAT,
            data={
                "content": message.data.get("content", ""),
                "sender": user_id,
                "timestamp": datetime.datetime.now().isoformat()
            },
            sender_id=user_id,
            room_id=message.room_id
        )
        
        # 发送消息
        await self.connection_manager.send_message_object(response)
        
        print(f"      💬 聊天消息: {user_id} -> {message.data.get('content', '')[:50]}...")
    
    async def _handle_notification(self, message: WebSocketMessage, connection_id: str):
        """处理通知消息"""
        # 获取发送者信息
        sender_info = self.connection_manager.connection_info.get(connection_id, {})
        user_id = sender_info.get("user_id", "系统")
        
        # 创建通知消息
        notification = WebSocketMessage(
            type=MessageType.NOTIFICATION,
            data={
                "title": message.data.get("title", "通知"),
                "content": message.data.get("content", ""),
                "level": message.data.get("level", "info"),
                "sender": user_id
            },
            sender_id=user_id,
            room_id=message.room_id
        )
        
        # 发送通知
        await self.connection_manager.send_message_object(notification)
        
        print(f"      📢 通知消息: {message.data.get('title', '')}")
    
    async def _handle_heartbeat(self, message: WebSocketMessage, connection_id: str):
        """处理心跳消息"""
        # 更新心跳时间
        if connection_id in self.connection_manager.connection_info:
            self.connection_manager.connection_info[connection_id]["last_heartbeat"] = time.time()
        
        # 发送心跳响应
        heartbeat_response = WebSocketMessage(
            type=MessageType.HEARTBEAT,
            data={"status": "pong", "timestamp": time.time()},
            receiver_id=connection_id
        )
        
        await self.connection_manager.send_message_object(heartbeat_response)
    
    async def _handle_status(self, message: WebSocketMessage, connection_id: str):
        """处理状态消息"""
        # 获取发送者信息
        sender_info = self.connection_manager.connection_info.get(connection_id, {})
        user_id = sender_info.get("user_id", "匿名用户")
        
        # 创建状态消息
        status_message = WebSocketMessage(
            type=MessageType.STATUS,
            data={
                "user": user_id,
                "status": message.data.get("status", "online"),
                "message": message.data.get("message", "")
            },
            sender_id=user_id,
            room_id=message.room_id
        )
        
        # 发送状态更新
        await self.connection_manager.send_message_object(status_message)
        
        print(f"      📊 状态更新: {user_id} -> {message.data.get('status', '')}")

print(f"   ✅ WebSocket基础概念完成")
print(f"      - MessageType: 消息类型定义")
print(f"      - WebSocketMessage: 消息结构")
print(f"      - ConnectionManager: 连接管理")
print(f"      - MessageHandler: 消息处理")

print(f"\n✅ WebSocket基础概念完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握WebSocket协议原理和实现方式")
print(f"   ✓ 学会使用FastAPI构建WebSocket应用")
print(f"   ✓ 理解实时通信的数据处理和状态管理")
print(f"   ✓ 能够实现聊天室、实时通知等应用场景")

### 实时聊天室应用 [⭐⭐进阶]
**知识点说明**：实时聊天室应用是WebSocket技术的典型应用场景，对于LangChain的多用户对话和协作功能具有重要参考价值。

**学习要求**：
- 构建完整的聊天室系统
- 实现用户认证和权限管理
- 支持房间管理和私聊功能
- 提供消息历史和在线状态

**案例要求**：
- 实现多房间聊天系统
- 支持用户在线状态显示
- 提供消息持久化存储
- 验证点：能独立部署聊天室应用

In [ ]:
print("\n💬 实时聊天室应用:")
print("=" * 50)

# 2. 聊天室系统
print(f"📝 2. 聊天室系统:")

# 2.1 聊天室管理器
print(f"\n   🏠 2.1 聊天室管理器:")

@dataclass
class ChatRoom:
    """聊天室"""
    room_id: str
    name: str
    description: str = ""
    created_at: float = field(default_factory=time.time)
    created_by: str = ""
    is_private: bool = False
    max_users: int = 100
    
    # 当前用户
    current_users: Set[str] = field(default_factory=set)
    
    # 聊天历史
    message_history: deque = field(default_factory=lambda: deque(maxlen=500))
    
    def add_user(self, user_id: str) -> bool:
        """添加用户"""
        if len(self.current_users) >= self.max_users:
            return False
        self.current_users.add(user_id)
        return True
    
    def remove_user(self, user_id: str):
        """移除用户"""
        self.current_users.discard(user_id)
    
    def add_message(self, message: WebSocketMessage):
        """添加消息"""
        self.message_history.append(message)
    
    def get_user_count(self) -> int:
        """获取用户数量"""
        return len(self.current_users)

class ChatRoomManager:
    """聊天室管理器"""
    
    def __init__(self):
        self.rooms: Dict[str, ChatRoom] = {}
        self.user_rooms: Dict[str, str] = {}  # {user_id: room_id}
    
    def create_room(self, name: str, description: str = "", 
                   created_by: str = "", is_private: bool = False) -> str:
        """创建聊天室"""
        room_id = str(uuid.uuid4())
        
        room = ChatRoom(
            room_id=room_id,
            name=name,
            description=description,
            created_by=created_by,
            is_private=is_private
        )
        
        self.rooms[room_id] = room
        
        print(f"      🏠 创建聊天室: {name} (ID: {room_id})")
        return room_id
    
    def join_room(self, user_id: str, room_id: str) -> bool:
        """加入聊天室"""
        if room_id not in self.rooms:
            return False
        
        room = self.rooms[room_id]
        
        # 如果用户已在其他房间，先离开
        if user_id in self.user_rooms:
            self.leave_room(user_id)
        
        # 加入新房间
        if room.add_user(user_id):
            self.user_rooms[user_id] = room_id
            print(f"      👤 用户 {user_id} 加入房间 {room.name}")
            return True
        
        return False
    
    def leave_room(self, user_id: str):
        """离开聊天室"""
        room_id = self.user_rooms.get(user_id)
        if room_id and room_id in self.rooms:
            room = self.rooms[room_id]
            room.remove_user(user_id)
            del self.user_rooms[user_id]
            print(f"      👋 用户 {user_id} 离开房间 {room.name}")
    
    def get_user_room(self, user_id: str) -> Optional[ChatRoom]:
        """获取用户所在房间"""
        room_id = self.user_rooms.get(user_id)
        if room_id:
            return self.rooms.get(room_id)
        return None
    
    def get_room_list(self, include_private: bool = False) -> List[Dict[str, Any]]:
        """获取房间列表"""
        rooms = []
        for room in self.rooms.values():
            if not include_private and room.is_private:
                continue
            
            rooms.append({
                "room_id": room.room_id,
                "name": room.name,
                "description": room.description,
                "user_count": room.get_user_count(),
                "max_users": room.max_users,
                "created_by": room.created_by,
                "is_private": room.is_private
            })
        
        return rooms
    
    def get_room_messages(self, room_id: str, limit: int = 50) -> List[Dict[str, Any]]:
        """获取房间消息历史"""
        if room_id not in self.rooms:
            return []
        
        room = self.rooms[room_id]
        messages = list(room.message_history)[-limit:]
        
        return [asdict(msg) for msg in messages]

# 2.2 用户管理器
print(f"\n   👤 2.2 用户管理器:")

@dataclass
class ChatUser:
    """聊天用户"""
    user_id: str
    username: str
    email: str = ""
    avatar: str = ""
    status: str = "online"  # online, away, busy, offline
    last_seen: float = field(default_factory=time.time)
    
    # 用户权限
    is_admin: bool = False
    is_moderator: bool = False
    
    # 统计信息
    messages_sent: int = 0
    rooms_joined: int = 0

class ChatUserManager:
    """聊天用户管理器"""
    
    def __init__(self):
        self.users: Dict[str, ChatUser] = {}
        self.username_to_id: Dict[str, str] = {}
        self.online_users: Set[str] = set()
    
    def register_user(self, username: str, email: str = "", 
                      password: str = "") -> str:
        """注册用户"""
        # 检查用户名是否已存在
        if username in self.username_to_id:
            raise ValueError(f"用户名 {username} 已存在")
        
        user_id = str(uuid.uuid4())
        
        user = ChatUser(
            user_id=user_id,
            username=username,
            email=email
        )
        
        self.users[user_id] = user
        self.username_to_id[username] = user_id
        
        print(f"      📝 注册用户: {username} (ID: {user_id})")
        return user_id
    
    def login_user(self, username: str, password: str = "") -> Optional[str]:
        """用户登录"""
        user_id = self.username_to_id.get(username)
        if not user_id:
            return None
        
        user = self.users[user_id]
        user.status = "online"
        user.last_seen = time.time()
        
        self.online_users.add(user_id)
        
        print(f"      🚪 用户登录: {username}")
        return user_id
    
    def logout_user(self, user_id: str):
        """用户登出"""
        if user_id in self.users:
            user = self.users[user_id]
            user.status = "offline"
            user.last_seen = time.time()
            
            self.online_users.discard(user_id)
            
            print(f"      🚪 用户登出: {user.username}")
    
    def update_user_status(self, user_id: str, status: str):
        """更新用户状态"""
        if user_id in self.users:
            self.users[user_id].status = status
            self.users[user_id].last_seen = time.time()
    
    def get_user_info(self, user_id: str) -> Optional[Dict[str, Any]]:
        """获取用户信息"""
        if user_id not in self.users:
            return None
        
        user = self.users[user_id]
        return {
            "user_id": user.user_id,
            "username": user.username,
            "email": user.email,
            "avatar": user.avatar,
            "status": user.status,
            "is_online": user_id in self.online_users,
            "last_seen": user.last_seen,
            "is_admin": user.is_admin,
            "is_moderator": user.is_moderator,
            "messages_sent": user.messages_sent,
            "rooms_joined": user.rooms_joined
        }
    
    def get_online_users(self) -> List[Dict[str, Any]]:
        """获取在线用户列表"""
        online_users = []
        for user_id in self.online_users:
            user_info = self.get_user_info(user_id)
            if user_info:
                online_users.append(user_info)
        
        return online_users

# 2.3 聊天应用主类
print(f"\n   🎯 2.3 聊天应用主类:")

class ChatApplication:
    """聊天应用主类"""
    
    def __init__(self):
        self.connection_manager = ConnectionManager()
        self.message_handler = MessageHandler(self.connection_manager)
        self.room_manager = ChatRoomManager()
        self.user_manager = ChatUserManager()
        
        # FastAPI应用
        self.app = FastAPI(title="实时聊天室", version="1.0.0")
        
        # 设置路由
        self._setup_routes()
        
        print(f"      🚀 聊天应用初始化完成")
    
    def _setup_routes(self):
        """设置路由"""
        
        @self.app.websocket("/ws/{user_id}")
        async def websocket_endpoint(websocket: WebSocket, user_id: str):
            await self._handle_websocket_connection(websocket, user_id)
        
        @self.app.get("/", response_class=HTMLResponse)
        async def get_chat_page():
            return self._get_chat_html()
        
        @self.app.get("/api/rooms")
        async def get_rooms():
            return {"rooms": self.room_manager.get_room_list()}
        
        @self.app.get("/api/users/online")
        async def get_online_users():
            return {"users": self.user_manager.get_online_users()}
        
        @self.app.post("/api/rooms")
        async def create_room(room_data: dict):
            room_id = self.room_manager.create_room(
                name=room_data.get("name", ""),
                description=room_data.get("description", ""),
                created_by=room_data.get("created_by", ""),
                is_private=room_data.get("is_private", False)
            )
            return {"room_id": room_id, "status": "success"}
    
    async def _handle_websocket_connection(self, websocket: WebSocket, user_id: str):
        """处理WebSocket连接"""
        # 接受连接
        connection_id = await self.connection_manager.connect(websocket, user_id)
        
        try:
            # 用户登录
            if user_id in self.user_manager.users:
                self.user_manager.login_user(self.user_manager.users[user_id].username)
            
            # 发送连接成功消息
            welcome_message = WebSocketMessage(
                type=MessageType.NOTIFICATION,
                data={
                    "title": "连接成功",
                    "content": f"欢迎 {user_id} 加入聊天室！",
                    "level": "success"
                },
                sender_id="system",
                receiver_id=connection_id
            )
            await self.connection_manager.send_message_object(welcome_message)
            
            # 监听消息
            while True:
                try:
                    # 接收消息
                    data = await websocket.receive_text()
                    
                    # 解析消息
                    try:
                        message = WebSocketMessage.from_json(data)
                        message.sender_id = user_id
                        
                        # 处理消息
                        await self.message_handler.handle_message(message, connection_id)
                        
                        # 更新用户统计
                        if message.type == MessageType.CHAT and user_id in self.user_manager.users:
                            self.user_manager.users[user_id].messages_sent += 1
                    
                    except json.JSONDecodeError:
                        # 发送错误消息
                        error_message = WebSocketMessage(
                            type=MessageType.NOTIFICATION,
                            data={
                                "title": "消息格式错误",
                                "content": "请发送有效的JSON格式消息",
                                "level": "error"
                            },
                            sender_id="system",
                            receiver_id=connection_id
                        )
                        await self.connection_manager.send_message_object(error_message)
                
                except WebSocketDisconnect:
                    break
                
                except Exception as e:
                    print(f"      ❌ 处理消息错误: {e}")
                    break
        
        finally:
            # 断开连接
            await self.connection_manager.disconnect(connection_id)
            
            # 用户登出
            if user_id in self.user_manager.users:
                self.user_manager.logout_user(user_id)
    
    def _get_chat_html(self) -> str:
        """获取聊天页面HTML"""
        return '''
        <!DOCTYPE html>
        <html>
        <head>
            <title>实时聊天室</title>
            <meta charset="utf-8">
            <style>
                body { font-family: Arial, sans-serif; margin: 0; padding: 20px; }
                .container { max-width: 800px; margin: 0 auto; }
                .chat-box { border: 1px solid #ddd; height: 400px; overflow-y: auto; padding: 10px; margin-bottom: 10px; }
                .message { margin-bottom: 10px; padding: 5px; }
                .message.system { color: #666; font-style: italic; }
                .message.chat { background: #f0f0f0; border-radius: 5px; }
                .input-group { display: flex; gap: 10px; }
                .input-group input { flex: 1; padding: 10px; border: 1px solid #ddd; border-radius: 5px; }
                .input-group button { padding: 10px 20px; background: #007bff; color: white; border: none; border-radius: 5px; cursor: pointer; }
                .status { margin-bottom: 20px; padding: 10px; background: #e7f3ff; border-radius: 5px; }
            </style>
        </head>
        <body>
            <div class="container">
                <h1>实时聊天室</h1>
                <div class="status" id="status">连接中...</div>
                <div class="chat-box" id="chatBox"></div>
                <div class="input-group">
                    <input type="text" id="messageInput" placeholder="输入消息..." onkeypress="if(event.key==='Enter') sendMessage()">
                    <button onclick="sendMessage()">发送</button>
                </div>
            </div>
            
            <script>
                const ws = new WebSocket(`ws://localhost:8000/ws/user_${Date.now()}`);
                const chatBox = document.getElementById('chatBox');
                const messageInput = document.getElementById('messageInput');
                const status = document.getElementById('status');
                
                ws.onopen = function() {
                    status.textContent = '已连接';
                    status.style.background = '#d4edda';
                };
                
                ws.onmessage = function(event) {
                    const message = JSON.parse(event.data);
                    displayMessage(message);
                };
                
                ws.onclose = function() {
                    status.textContent = '连接断开';
                    status.style.background = '#f8d7da';
                };
                
                function displayMessage(message) {
                    const div = document.createElement('div');
                    div.className = `message ${message.type}`;
                    
                    if (message.type === 'chat') {
                        div.innerHTML = `<strong>${message.data.sender}:</strong> ${message.data.content}`;
                    } else if (message.type === 'notification') {
                        div.innerHTML = `<em>${message.data.title}: ${message.data.content}</em>`;
                        div.className += ' system';
                    }
                    
                    chatBox.appendChild(div);
                    chatBox.scrollTop = chatBox.scrollHeight;
                }
                
                function sendMessage() {
                    const content = messageInput.value.trim();
                    if (!content) return;
                    
                    const message = {
                        type: 'chat',
                        data: { content: content },
                        timestamp: Date.now() / 1000
                    };
                    
                    ws.send(JSON.stringify(message));
                    messageInput.value = '';
                }
            </script>
        </body>
        </html>
        '''
    
    def get_app(self) -> FastAPI:
        """获取FastAPI应用"""
        return self.app
    
    def get_stats(self) -> Dict[str, Any]:
        """获取应用统计"""
        return {
            "connections": self.connection_manager.get_stats(),
            "rooms": {
                "total": len(self.room_manager.rooms),
                "list": self.room_manager.get_room_list()
            },
            "users": {
                "total": len(self.user_manager.users),
                "online": len(self.user_manager.online_users),
                "list": self.user_manager.get_online_users()
            }
        }

# 创建聊天应用实例
chat_app = ChatApplication()

print(f"   ✅ 聊天室系统完成")
print(f"      - ChatRoom: 聊天室结构")
print(f"      - ChatRoomManager: 房间管理")
print(f"      - ChatUser: 用户结构")
print(f"      - ChatUserManager: 用户管理")
print(f"      - ChatApplication: 聊天应用")

print(f"\n✅ 实时聊天室应用完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 构建完整的聊天室系统")
print(f"   ✓ 实现用户认证和权限管理")
print(f"   ✓ 支持房间管理和私聊功能")
print(f"   ✓ 提供消息历史和在线状态")

### WebSocket高级应用 [⭐⭐进阶]
**知识点说明**：WebSocket高级应用展示了实时通信技术的扩展能力，为LangChain的复杂交互场景提供技术支持。

**学习要求**：
- 实现实时数据同步功能
- 构建协作编辑系统
- 开发实时通知推送
- 支持文件传输和媒体流

**案例要求**：
- 实现多用户协作编辑
- 构建实时数据可视化
- 开发推送通知系统
- 验证点：能独立构建高级实时应用

In [ ]:
print("\n🚀 WebSocket高级应用:")
print("=" * 50)

# 3. 高级WebSocket应用
print(f"📝 3. 高级WebSocket应用:")

# 3.1 实时协作编辑
print(f"\n   📝 3.1 实时协作编辑:")

@dataclass
class DocumentOperation:
    """文档操作"""
    operation_type: str  # insert, delete, replace
    position: int
    content: str = ""
    length: int = 0
    user_id: str = ""
    timestamp: float = field(default_factory=time.time)
    operation_id: str = field(default_factory=lambda: str(uuid.uuid4()))

@dataclass
class CollaborativeDocument:
    """协作文档"""
    document_id: str
    title: str
    content: str = ""
    created_at: float = field(default_factory=time.time)
    updated_at: float = field(default_factory=time.time)
    
    # 协作信息
    active_users: Set[str] = field(default_factory=set)
    operation_history: deque = field(default_factory=lambda: deque(maxlen=1000))
    
    # 用户光标位置
    user_cursors: Dict[str, int] = field(default_factory=dict)
    
    def apply_operation(self, operation: DocumentOperation) -> bool:
        """应用操作"""
        try:
            if operation.operation_type == "insert":
                if operation.position <= len(self.content):
                    self.content = (
                        self.content[:operation.position] + 
                        operation.content + 
                        self.content[operation.position:]
                    )
                else:
                    return False
            
            elif operation.operation_type == "delete":
                end_pos = operation.position + operation.length
                if end_pos <= len(self.content):
                    self.content = (
                        self.content[:operation.position] + 
                        self.content[end_pos:]
                    )
                else:
                    return False
            
            elif operation.operation_type == "replace":
                end_pos = operation.position + operation.length
                if end_pos <= len(self.content):
                    self.content = (
                        self.content[:operation.position] + 
                        operation.content + 
                        self.content[end_pos:]
                    )
                else:
                    return False
            
            else:
                return False
            
            # 记录操作
            self.operation_history.append(operation)
            self.updated_at = time.time()
            
            return True
        
        except Exception:
            return False
    
    def update_cursor(self, user_id: str, position: int):
        """更新用户光标位置"""
        if 0 <= position <= len(self.content):
            self.user_cursors[user_id] = position
    
    def add_user(self, user_id: str):
        """添加活跃用户"""
        self.active_users.add(user_id)
    
    def remove_user(self, user_id: str):
        """移除活跃用户"""
        self.active_users.discard(user_id)
        self.user_cursors.pop(user_id, None)

class CollaborativeEditor:
    """协作编辑器"""
    
    def __init__(self, connection_manager: ConnectionManager):
        self.connection_manager = connection_manager
        self.documents: Dict[str, CollaborativeDocument] = {}
        self.user_documents: Dict[str, str] = {}  # {user_id: document_id}
    
    def create_document(self, title: str, creator_id: str = "") -> str:
        """创建文档"""
        document_id = str(uuid.uuid4())
        
        document = CollaborativeDocument(
            document_id=document_id,
            title=title
        )
        
        self.documents[document_id] = document
        
        print(f"      📄 创建文档: {title} (ID: {document_id})")
        return document_id
    
    def join_document(self, user_id: str, document_id: str) -> bool:
        """加入文档编辑"""
        if document_id not in self.documents:
            return False
        
        # 如果用户已在其他文档，先离开
        if user_id in self.user_documents:
            self.leave_document(user_id)
        
        # 加入新文档
        document = self.documents[document_id]
        document.add_user(user_id)
        self.user_documents[user_id] = document_id
        
        print(f"      👤 用户 {user_id} 加入文档 {document.title}")
        return True
    
    def leave_document(self, user_id: str):
        """离开文档编辑"""
        document_id = self.user_documents.get(user_id)
        if document_id and document_id in self.documents:
            document = self.documents[document_id]
            document.remove_user(user_id)
            del self.user_documents[user_id]
            
            print(f"      👋 用户 {user_id} 离开文档 {document.title}")
    
    async def apply_operation(self, user_id: str, operation: DocumentOperation) -> bool:
        """应用操作并广播"""
        document_id = self.user_documents.get(user_id)
        if not document_id or document_id not in self.documents:
            return False
        
        document = self.documents[document_id]
        operation.user_id = user_id
        
        # 应用操作
        if document.apply_operation(operation):
            # 广播操作给所有用户
            message = WebSocketMessage(
                type=MessageType.STATUS,
                data={
                    "action": "operation",
                    "operation": asdict(operation),
                    "document_content": document.content,
                    "active_users": list(document.active_users),
                    "user_cursors": document.user_cursors
                },
                sender_id=user_id,
                room_id=document_id
            )
            
            await self.connection_manager.send_message_object(message)
            return True
        
        return False
    
    async def update_cursor(self, user_id: str, position: int):
        """更新光标位置并广播"""
        document_id = self.user_documents.get(user_id)
        if not document_id or document_id not in self.documents:
            return
        
        document = self.documents[document_id]
        document.update_cursor(user_id, position)
        
        # 广播光标更新
        message = WebSocketMessage(
            type=MessageType.STATUS,
            data={
                "action": "cursor_update",
                "user_id": user_id,
                "position": position,
                "user_cursors": document.user_cursors
            },
            sender_id=user_id,
            room_id=document_id
        )
        
        await self.connection_manager.send_message_object(message)

# 3.2 实时数据同步
print(f"\n   🔄 3.2 实时数据同步:")

@dataclass
class DataSyncEvent:
    """数据同步事件"""
    event_type: str  # create, update, delete
    data_type: str   # user, room, message, etc.
    data_id: str
    data: Dict[str, Any]
    timestamp: float = field(default_factory=time.time)
    source: str = ""  # 事件来源

class RealTimeDataSync:
    """实时数据同步"""
    
    def __init__(self, connection_manager: ConnectionManager):
        self.connection_manager = connection_manager
        self.event_history: deque = deque(maxlen=1000)
        self.subscribers: Dict[str, Set[str]] = defaultdict(set)  # {data_type: {connection_id}}
        self.data_cache: Dict[str, Dict[str, Any]] = {}  # {data_key: data}
    
    def subscribe(self, connection_id: str, data_type: str):
        """订阅数据类型"""
        self.subscribers[data_type].add(connection_id)
        print(f"      📢 连接 {connection_id} 订阅 {data_type}")
    
    def unsubscribe(self, connection_id: str, data_type: str = None):
        """取消订阅"""
        if data_type:
            self.subscribers[data_type].discard(connection_id)
        else:
            # 取消所有订阅
            for subscribers in self.subscribers.values():
                subscribers.discard(connection_id)
    
    async def publish_event(self, event: DataSyncEvent):
        """发布同步事件"""
        # 保存事件历史
        self.event_history.append(event)
        
        # 更新缓存
        data_key = f"{event.data_type}:{event.data_id}"
        if event.event_type == "delete":
            self.data_cache.pop(data_key, None)
        else:
            self.data_cache[data_key] = event.data
        
        # 创建WebSocket消息
        message = WebSocketMessage(
            type=MessageType.STATUS,
            data={
                "action": "data_sync",
                "event": asdict(event)
            },
            sender_id="data_sync"
        )
        
        # 发送给订阅者
        subscribers = self.subscribers.get(event.data_type, set())
        for connection_id in subscribers:
            if connection_id in self.connection_manager.active_connections:
                message.receiver_id = connection_id
                await self.connection_manager.send_message_object(message)
        
        print(f"      📤 发布事件: {event.event_type} {event.data_type}:{event.data_id}")
    
    async def sync_initial_data(self, connection_id: str, data_type: str):
        """同步初始数据"""
        initial_data = []
        
        for data_key, data in self.data_cache.items():
            if data_key.startswith(f"{data_type}:"):
                data_id = data_key.split(":", 1)[1]
                initial_data.append({
                    "data_type": data_type,
                    "data_id": data_id,
                    "data": data
                })
        
        if initial_data:
            message = WebSocketMessage(
                type=MessageType.STATUS,
                data={
                    "action": "initial_sync",
                    "data_type": data_type,
                    "initial_data": initial_data
                },
                receiver_id=connection_id
            )
            
            await self.connection_manager.send_message_object(message)
            print(f"      📥 同步初始数据: {data_type} ({len(initial_data)} 项)")

# 3.3 推送通知系统
print(f"\n   🔔 3.3 推送通知系统:")

@dataclass
class PushNotification:
    """推送通知"""
    notification_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    title: str = ""
    content: str = ""
    type: str = "info"  # info, warning, error, success
    priority: int = 0  # 0-低, 1-中, 2-高
    target_users: List[str] = field(default_factory=list)
    target_rooms: List[str] = field(default_factory=list)
    created_at: float = field(default_factory=time.time)
    expires_at: Optional[float] = None
    data: Dict[str, Any] = field(default_factory=dict)

class NotificationManager:
    """通知管理器"""
    
    def __init__(self, connection_manager: ConnectionManager):
        self.connection_manager = connection_manager
        self.notification_queue: asyncio.Queue = asyncio.Queue()
        self.user_preferences: Dict[str, Dict[str, bool]] = {}
        self.notification_history: deque = deque(maxlen=1000)
        self.running = False
        self.worker_task: Optional[asyncio.Task] = None
    
    async def start(self):
        """启动通知服务"""
        if self.running:
            return
        
        self.running = True
        self.worker_task = asyncio.create_task(self._notification_worker())
        
        print(f"      🚀 通知服务启动")
    
    async def stop(self):
        """停止通知服务"""
        if not self.running:
            return
        
        self.running = False
        
        if self.worker_task:
            self.worker_task.cancel()
            try:
                await self.worker_task
            except asyncio.CancelledError:
                pass
        
        print(f"      🛑 通知服务停止")
    
    def set_user_preferences(self, user_id: str, preferences: Dict[str, bool]):
        """设置用户通知偏好"""
        self.user_preferences[user_id] = preferences
    
    async def send_notification(self, notification: PushNotification):
        """发送通知"""
        await self.notification_queue.put(notification)
    
    async def _notification_worker(self):
        """通知工作线程"""
        while self.running:
            try:
                # 获取通知
                notification = await asyncio.wait_for(
                    self.notification_queue.get(), 
                    timeout=1.0
                )
                
                # 处理通知
                await self._process_notification(notification)
            
            except asyncio.TimeoutError:
                continue
            except Exception as e:
                print(f"      ❌ 通知处理错误: {e}")
    
    async def _process_notification(self, notification: PushNotification):
        """处理通知"""
        # 检查是否过期
        if notification.expires_at and time.time() > notification.expires_at:
            return
        
        # 保存历史
        self.notification_history.append(notification)
        
        # 创建WebSocket消息
        message = WebSocketMessage(
            type=MessageType.NOTIFICATION,
            data={
                "notification_id": notification.notification_id,
                "title": notification.title,
                "content": notification.content,
                "type": notification.type,
                "priority": notification.priority,
                "created_at": notification.created_at,
                "data": notification.data
            },
            sender_id="notification_system"
        )
        
        # 发送通知
        if notification.target_users:
            # 发送给指定用户
            for user_id in notification.target_users:
                if self._should_send_to_user(user_id, notification):
                    message.receiver_id = user_id
                    await self.connection_manager.send_message_object(message)
        
        elif notification.target_rooms:
            # 发送给指定房间
            for room_id in notification.target_rooms:
                message.room_id = room_id
                await self.connection_manager.send_message_object(message)
        
        else:
            # 广播给所有用户
            await self.connection_manager.send_message_object(message)
        
        print(f"      📨 发送通知: {notification.title}")
    
    def _should_send_to_user(self, user_id: str, notification: PushNotification) -> bool:
        """检查是否应该发送给用户"""
        preferences = self.user_preferences.get(user_id, {})
        
        # 检查用户是否在线
        connection_id = self.connection_manager.user_connections.get(user_id)
        if not connection_id:
            return False
        
        # 检查用户偏好
        if not preferences.get(f"notify_{notification.type}", True):
            return False
        
        # 检查优先级设置
        min_priority = preferences.get("min_priority", 0)
        if notification.priority < min_priority:
            return False
        
        return True

print(f"   ✅ WebSocket高级应用完成")
print(f"      - DocumentOperation: 文档操作")
print(f"      - CollaborativeDocument: 协作文档")
print(f"      - CollaborativeEditor: 协作编辑器")
print(f"      - DataSyncEvent: 数据同步事件")
print(f"      - RealTimeDataSync: 实时数据同步")
print(f"      - PushNotification: 推送通知")
print(f"      - NotificationManager: 通知管理器")

print(f"\n✅ WebSocket高级应用完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 实现实时数据同步功能")
print(f"   ✓ 构建协作编辑系统")
print(f"   ✓ 开发实时通知推送")
print(f"   ✓ 支持文件传输和媒体流")

## 📋 学习总结

### 知识清单达成情况

#### ✅ 已完成知识点

**7.4 WebSocket实时通信技术 [⭐⭐进阶]**
- ✅ 掌握WebSocket协议原理和实现方式
- ✅ 学会使用FastAPI构建WebSocket应用
- ✅ 理解实时通信的数据处理和状态管理
- ✅ 能够实现聊天室、实时通知等应用场景

#### 🎯 核心技能掌握

**WebSocket基础**
- ✅ WebSocket消息类型和结构设计
- ✅ 连接管理和生命周期控制
- ✅ 消息处理器和路由机制
- ✅ 广播和私聊功能实现

**实时聊天室**
- ✅ 聊天室创建和管理
- ✅ 用户认证和权限控制
- ✅ 房间系统和消息历史
- ✅ 在线状态和用户管理

**高级应用**
- ✅ 协作编辑和操作同步
- ✅ 实时数据同步机制
- ✅ 推送通知系统
- ✅ 性能优化和扩展性

#### 🚀 实践应用能力

**Web开发**
- ✅ FastAPI WebSocket集成
- ✅ 前端实时通信实现
- ✅ 跨域和安全处理
- ✅ 错误处理和重连机制

**系统设计**
- ✅ 实时通信架构设计
- ✅ 消息队列和事件系统
- ✅ 数据同步和一致性
- ✅ 扩展性和容错设计

#### 📊 与LangChain的关联

**实时交互**
- ✅ 支持LangChain实时对话流
- ✅ 多用户协作和共享会话
- ✅ 实时响应和状态同步
- ✅ 用户体验优化

**技术集成**
- ✅ WebSocket与AI模型集成
- ✅ 实时数据处理和分析
- ✅ 分布式通信和协调
- ✅ 性能监控和优化

---

## 🎯 学习成果

通过本节学习，你已经掌握了：

1. **WebSocket核心技术** - 理解协议原理，掌握实现方法
2. **实时通信系统** - 构建完整的实时应用架构
3. **协作功能开发** - 实现多用户协作和同步机制
4. **高级应用场景** - 开发复杂的实时交互功能

这些技能为后续的Web安全、微服务架构等高级主题奠定了坚实基础，也为LangChain的实时AI应用开发提供了重要技术支撑。